In [1]:
import os
import pandas as pd
path = '/home/lucifer/Data/summit-data-pseudonymized/'
def make_csv(path,lis):
    
    main_df = {'date-hour':[],'login-node':[],'wusers':[],'memory-used':[],'running-process':[],'cpu_usage %':[],'running-jobs':[],'total-jobs':[],'time-to-ls':[],'time-for-1G':[],'df-usage':[]}
    
    for i in lis:
        
        
        temp_var = i.split('.')
        
        #main_df['login-node'].append(temp_var[0])
        if 'Jan08_2020' in i:
            continue
        
        f = open(path+'/'+i,'r')
        
        
        while True:
            x = f.readline().strip()
        
        
            main_df['login-node'].append(temp_var[0])
            if 'Hour' in x:
                hr = x.split(' ')
                main_df['date-hour'].append(temp_var[-2]+'-'+hr[-1])
        
        # calculating number of users logged-in at that time.
        
            users = []
            while x !='w --':
                x = f.readline().strip()
            
            x = f.readline().strip()
        
            while True:
                if x == 'endw --':
                    break
                u = list(filter(None,x.split(' ')))
                users.append(u[0])
                x = f.readline().strip()
        
            main_df['wusers'].append(len(set(users))- 2)
            
        # end of calculating the  number of logged users at that time.
        
        # calculating the memory used by the process at that time.
            x = f.readline().strip()
            while x != 'meminfo --':
                x = f.readline.strip()
            memory = int(f.readline().split(' ')[-2])
            used = memory - int(f.readline().split(' ')[-2])
            main_df['memory-used'].append(round(used/1048576,2))
        
            while x!= 'endvmstat --':
                x = f.readline().strip()
            
        # end of calculating the memory used by the process at that time.
        
        # calculating the running process by ps aux.
            x = f.readline().strip()
            while x != 'ps aux --':
                x = f.readline().strip()
            
            process = 0
            f.readline()
            while x != 'endps aux --':
                x = f.readline().strip()
                process += 1
        
            main_df['running-process'].append(process)
           
        # end of calculating the running process by ps aux.
        
        # calculating the cpu usage of the process at that time.
            cpu_usage = 0
            flag = False
        
            while x != 'top -n 1 -bc | awk !~/u1776/ --':
                x = f.readline().strip()
            
            while True:
                if x == 'endtop -n 1 -bc | awk !~/u1776/ --':
                    break
                if flag:
                    a = list(filter(None,x.split(' ')))
                
                #print(a)
                    cpu_usage += float(a[8])
                if 'PID USER' in x:
                    flag = True
                x = f.readline().strip()
            
            main_df['cpu_usage %'].append(round(cpu_usage,2))
            
        # end of the cpu usage block to calculate the total cpu usage at that time.
        
        # calculating the total jobs and running jobs at that time.
            flag = False
            jobs = 0
            running = 0
            while x != 'bjobs -a -u all --':
                x = f.readline().strip()
            
            while True:
                if x == 'endbjobs -a -u all --':
                    break
                if flag:
                    a = list(filter(None,x.split(' ')))
                    if a[2] == "RUN":
                        running += 1
                    jobs+=1
                if 'JOBID   USER' in x:
                    flag = True
                x = f.readline().strip()
            main_df['running-jobs'].append(running)
            main_df['total-jobs'].append(jobs)
            
        # end of calculating total and running jobs at that time.
        
        # calculating the time to ls unaliased at that time.
            
            time = 0
            while x != 'home response time unaliased ls --':
                x = f.readline().strip()
            
            while True:
                if x == 'endhome response time colored ls':
                    break
            
                if 'real' in x:
                    time = x.split('	')[-1]
                    
                x = f.readline().strip()
            
            main_df['time-to-ls'].append(time)
           
        # end of time to unaliased ls in the system at that time.
        
        # time to create 1GB file in the system.
            flag = False
            counter = 1
            time = 0
            fail = False
            while x != 'gpfs scratch response time to create a 1 G file --':
                x = f.readline().strip()
            counter = 0
            while True:
                if x == 'endgpfs scratch response time to create a 1 G file --':
                    break
                if x =='':
                    counter +=1
                if counter > 1:
                    break
                if x =='fail':
                    fail = True
                    break
                    
                if 'real' in x and flag == False:
                    time = x.split('	')[-1]
                    flag = True
                x = f.readline().strip()
                
            if fail:
                main_df['time-for-1G'].append('-')
            else:
                main_df['time-for-1G'].append(time)
           
        # end of time to create a 1G file.
        
        # checking the df of cpu at that time.
            flag = False
            usage = 0
            if x == '':
                main_df['df-usage'].append('-')
            else:
                while x != 'df --':
                    x = f.readline().strip()
                
                f.readline()
                while True:
                    if x == 'enddf --':
                        break
                    if flag:
                        a = list(filter(None,x.split()))
                        usage += int(a[4][:-1])
                    x = f.readline().strip()
                    flag = True

                main_df['df-usage'].append(usage)
            
                
        # end of checking the df at that time.
            if x != '':
                f.readline()
                f.readline()
                f.readline()
            x = f.tell()
            if f.readline() == '':
                
                break
            else:
                f.seek(x)

        f.close()

    df = pd.DataFrame(main_df)
    dest_path = r'/home/lucifer/SMC-Data_Challenge/csv_files/'
    
    df.to_csv(os.path.join(dest_path+path.split('/')[-1]+'.csv'),index=False)



lis = os.listdir(path)
x= 0
for i in lis:
    
    abc = os.listdir(path+i)
    make_csv(path+i,abc)
    